In [72]:
import pandas as pd
import os
import numpy as np
import random

base_path : str = os.path.dirname(os.getcwd())
CSV_PATH  : str = base_path + '\\csv'
SRC_PATH  : str = base_path + '\\src'
JSON_PATH : str = base_path + '\\json'

In [73]:
user_rating = pd.read_json(JSON_PATH + r'\USER_RATING_DATA.json')
user_rating

,USER_ID,RATED,NAN_RATED
0,5,"{'1': 2.0, '4': 2.0, '6': 3.0, '8': 4.0, '12':...","{'0': -1.0, '2': -1.0, '3': -1.0, '5': -1.0, '..."
1,12,"{'2': 2.0, '3': 5.0, '5': 4.0, '9': 1.0, '10':...","{'0': -1.0, '1': -1.0, '4': -1.0, '6': -1.0, '..."
2,19,"{'0': 5.0, '1': 2.0, '5': 2.0, '6': 5.0, '10':...","{'2': -1.0, '3': -1.0, '4': -1.0, '7': -1.0, '..."
3,24,"{'1': 5.0, '4': 5.0, '5': 5.0, '6': 5.0, '8': ...","{'0': -1.0, '2': -1.0, '3': -1.0, '7': -1.0, '..."
4,31,"{'1': 0.0, '6': 3.0, '7': 1.0, '10': 0.0, '11'...","{'0': -1.0, '2': -1.0, '3': -1.0, '4': -1.0, '..."
...,...,...,...
353,1796,"{'0': 3.0, '1': 1.0, '3': 3.0, '6': 0.0, '7': ...","{'2': -1.0, '4': -1.0, '5': -1.0, '8': -1.0, '..."
354,1804,"{'1': 3.0, '2': 2.0, '3': 4.0, '9': 3.0, '10':...","{'0': -1.0, '4': -1.0, '5': -1.0, '6': -1.0, '..."
355,1813,"{'0': 3.0, '1': 5.0, '2': 5.0, '4': 3.0, '5': ...","{'3': -1.0, '7': -1.0, '8': -1.0, '9': -1.0, '..."
356,1815,"{'2': 4.0, '3': 3.0, '4': 2.0, '5': 1.0, '48':...","{'0': -1.0, '1': -1.0, '6': -1.0, '7': -1.0, '..."


In [74]:
movie_distance = pd.read_json(JSON_PATH + r'\movie_distance_graph.json')

In [256]:
def generate_weight_samples(num_samples=1000, weight_count=5, min_val=0.5, max_val=1.1, sum_constraint=None):
    import random
    samples = []
    
    for _ in range(num_samples):
        if sum_constraint is not None:
            # Generate weights that satisfy the sum constraint
            weights = []
            for _ in range(weight_count):
                weight = random.uniform(min_val, max_val)
                weights.append(weight)
            
            # Normalize to meet the sum constraint
            current_sum = sum(weights)
            if current_sum != 0:  # Avoid division by zero
                weights = [w * (sum_constraint / current_sum) for w in weights]
            
            # Clamp values to ensure they are within range after normalization
            weights = [max(min(w, max_val), min_val) for w in weights]
        else:
            # Generate weights freely within the range
            weights = [random.uniform(min_val, max_val) for _ in range(weight_count)]
        
        samples.append(weights)
    
    return samples

PARAMS_WEIGHTS = generate_weight_samples()

In [246]:
def decision_tree(movie_id, all_distances, training_ids, rating_data, threshold) -> int:

    """
        Dla movie_id wyszukujemyt filmy, które oddalone są od movie_id o mniej niż THRESHOLD.
        Jeśli taki jest, to zapamiętujemy jego ocenę, a potem ze wszystkich wyciątgamy średnią.
    """
    
    decisions: list = []

    for rated_movie in training_ids:

        feature_counter: int = 0

        for feat_dist_i, feature_distance in enumerate(all_distances[movie_id][int(rated_movie)]):

            if feature_distance < threshold[feat_dist_i]:

                feature_counter += 1

        if feature_counter == 5:

            decisions.append(rating_data[str(rated_movie)])


    return np.ceil(np.average(decisions)) if len(decisions) > 0 else None

def optimize_user(user, validate_ids, training_ids) -> tuple[float, list]:

    max_accuracy: float = -1.0
    best_weights: list = []

    for weights in PARAMS_WEIGHTS:

        accuracy: float = 0.0

        for validate_movie in validate_ids:

            # output = decision_tree(movie_id=validate_movie, all_distances=movie_distance, training_ids=training_ids, rating_data={k: v for k, v in user_rating["RATED"][user].items() if int(k) in [training_ids+validate_ids]}, threshold=weights)
            output = decision_tree(movie_id=validate_movie, all_distances=movie_distance, training_ids=training_ids, rating_data=user_rating["RATED"][user], threshold=weights)

            # print(f"Output: {output}, True value: {user_rating['RATED'][user][str(validate_movie)]}")

            if output is not None and output == user_rating['RATED'][user][str(validate_movie)]:
                accuracy += 1.0
        
        accuracy = accuracy/len(validate_ids)

        if accuracy > max_accuracy:

            max_accuracy = accuracy
            best_weights = weights
    
    return (max_accuracy, best_weights)

def test_user(user_id, best_weights, test_ids, train_ids) -> float:

    accuracy: float = 0.0

    for test_movie in test_ids:

        # output = decision_tree(movie_id=validate_movie, all_distances=movie_distance, training_ids=training_ids, rating_data={k: v for k, v in user_rating["RATED"][user].items() if int(k) in [training_ids+validate_ids]}, threshold=weights)
        output = decision_tree(movie_id=test_movie, all_distances=movie_distance, training_ids=train_ids, rating_data=user_rating["RATED"][user_id], threshold=best_weights)

        if output is not None and output == user_rating['RATED'][user_id][str(test_movie)]:
            accuracy += 1.0
    
    accuracy = accuracy/len(test_ids)

    return accuracy


In [ ]:
NUM_OF_CROSS_VALIDATION = 3
NUM_OF_TRAINED_USERS = len(user_rating)

list_best_weights_out: list = []

for user in range(NUM_OF_TRAINED_USERS):

    accuracy = 0
    best_k_out = 0
    best_weights_out = [1.1,1.1,1.1,1.1,1.1]

    for validation_id in range(NUM_OF_CROSS_VALIDATION):

        keys = list(user_rating['RATED'][user].keys())
        random.shuffle(keys)
        split_index = int(len(keys) * 0.9)
        train_valid_keys = keys[:split_index]
        test_keys = keys[split_index:]

        keys = list(train_valid_keys)
        random.shuffle(keys)
        split_index = int(len(keys) * 0.85)
        train_keys = keys[:split_index]
        valid_keys = keys[split_index:]


        test_ids = [int(key) for key in test_keys]
        validate_ids = [int(key) for key in valid_keys]
        training_ids = [int(key) for key in train_keys]

        # print(f'Train keys: {train_keys}\nValid keys: {valid_keys}\nTest keys: {test_keys}')

        _, temp_best_weights_out = optimize_user(user, validate_ids, training_ids)

        temp_accuracy = test_user(user_id=user, best_weights=temp_best_weights_out, test_ids=test_ids, train_ids=train_valid_keys)

        if temp_accuracy > accuracy:
            accuracy = temp_accuracy
            best_weights_out = temp_best_weights_out
        
    print(f"{user}. User: {user_rating['USER_ID'][user]} weights are: {best_weights_out} with appx accuracy of {accuracy:.2f}")
    list_best_weights_out.append(best_weights_out)



0. User: 5 weights are: [0.5350250120638428, 1.0314094592871403, 1.021467445345301, 1.0018618639886658, 0.7972512723147269] with appx accuracy of 0.89
1. User: 12 weights are: [0.6011306823927157, 1.0481301557228964, 1.048795083164943, 0.965307396103857, 0.9990078446994196] with appx accuracy of 0.44
2. User: 19 weights are: [0.793364932185679, 0.5455537568355044, 1.0375085300711517, 0.9670780120656732, 1.0025776790954009] with appx accuracy of 0.44
3. User: 24 weights are: [0.5350250120638428, 1.0314094592871403, 1.021467445345301, 1.0018618639886658, 0.7972512723147269] with appx accuracy of 0.78
4. User: 31 weights are: [1.0448665884293167, 0.5961561926715476, 1.0034179691770744, 0.9932570966126741, 1.06748678765139] with appx accuracy of 0.22
5. User: 52 weights are: [0.5350250120638428, 1.0314094592871403, 1.021467445345301, 1.0018618639886658, 0.7972512723147269] with appx accuracy of 0.33
6. User: 62 weights are: [0.8964587048433136, 0.6519319345678607, 0.6621500185975522, 1.059

KeyboardInterrupt: 

In [255]:
user_test_data = {
    user_rating['USER_ID'][i]: list_best_weights_out[i] for i in range(NUM_OF_TRAINED_USERS)
}
user_test_data

{5: [1.1, 1.1, 1.1, 1.1, 1.1],
 12: [0.5, 0.5, 1.1, 1.1, 0.5],
 19: [0.75, 1.1, 1.1, 1.1, 0.6],
 24: [1.1, 1.1, 1.1, 1.1, 1.1],
 31: [0.5, 0.5, 1.1, 1.1, 0.5],
 52: [1.1, 1.1, 1.1, 1.1, 1.1],
 62: [0.5, 0.5, 1.1, 1.1, 0.5],
 63: [0.5, 0.5, 1.1, 1.1, 0.5],
 68: [1.1, 1.1, 1.1, 1.1, 1.1],
 69: [0.5, 0.5, 1.1, 1.1, 0.5],
 70: [1.1, 1.1, 1.1, 1.1, 1.1],
 71: [0.75, 1.1, 1.1, 1.1, 0.6],
 78: [0.75, 1.1, 1.1, 1.1, 0.6],
 80: [0.75, 1.1, 1.1, 1.1, 0.6],
 90: [1.1, 1.1, 1.1, 1.1, 1.1],
 92: [0.75, 1.1, 1.1, 1.1, 0.6],
 93: [1.1, 1.1, 1.1, 1.1, 1.1],
 105: [1.1, 1.1, 1.1, 1.1, 1.1],
 106: [0.5, 0.5, 1.1, 1.1, 0.5],
 109: [0.5, 0.5, 1.1, 1.1, 0.5],
 118: [0.75, 1.1, 1.1, 1.1, 0.6],
 125: [0.75, 1.1, 1.1, 1.1, 0.6],
 131: [0.75, 1.1, 1.1, 1.1, 0.6],
 139: [0.75, 1.1, 1.1, 1.1, 0.6],
 140: [0.75, 1.1, 1.1, 1.1, 0.6],
 148: [0.75, 1.1, 1.1, 1.1, 0.6],
 149: [0.75, 1.1, 1.1, 1.1, 0.6],
 151: [0.75, 1.1, 1.1, 1.1, 0.6],
 154: [0.75, 1.1, 1.1, 1.1, 0.6],
 156: [0.75, 1.1, 1.1, 1.1, 0.6],
 165: [1.1, 1

In [252]:
user_test_data_df = pd.DataFrame(user_test_data)
user_test_data_df.to_json(JSON_PATH + '\\USER_HYPER_PARAMS.json', indent=4)

ValueError: All arrays must be of the same length

output = decision_tree(movie_id=2, all_distances=movie_distance, rating_data=user_rating.loc[user_rating["USER_ID"] == 5, "RATED"][0], threshold=[0.7, 0.7, 2, 2, 0.7])
output